In [31]:
import pandas as pd
import numpy as np

In [32]:
hist_data = pd.read_excel("C:/Users/danbi/Desktop/nfl_historical_data.xlsx")
hist_data

,Team,Date,Day,G#,Week,Seasonweek,Away?,Opp,Result,Cmp,...,RecBrkTkl,Yards Bef Contact,Yards After Contacts,RushBrkTkl,DADOT,Def Bltz,Def Bats,Def Hrry,QBKD,Def Prss
0,BUF,2022-09-08,Thu,1,1,20221,1.0,RAM,W,26,...,0,83,38,0,6.1,1,0,1,7,16
1,RAM,2022-09-08,Thu,1,1,20221,NaN,BUF,L,29,...,0,35,17,0,7.3,8,0,0,0,2
2,CRD,2022-09-11,Sun,1,1,20221,NaN,KAN,L,24,...,0,50,53,1,7.4,24,0,18,6,24
3,ATL,2022-09-11,Sun,1,1,20221,NaN,NOR,L,20,...,0,148,53,0,9.0,13,0,5,3,13
4,RAV,2022-09-11,Sun,1,1,20221,1.0,NYJ,W,17,...,2,56,7,1,4.5,19,1,6,6,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1625,SEA,2025-01-05,Sun,17,18,202418,1.0,RAM,W,20,...,0,94,32,4,4.8,13,0,0,5,9
1626,SFO,2025-01-05,Sun,17,18,202418,1.0,CRD,L,29,...,0,76,62,1,7.0,10,0,3,2,5
1627,TAM,2025-01-05,Sun,17,18,202418,NaN,NOR,W,21,...,1,134,45,3,8.3,20,0,2,3,7
1628,OTI,2025-01-05,Sun,17,18,202418,NaN,HTX,L,16,...,2,60,25,0,6.5,11,0,1,1,3


In [33]:
# zero imputation
hist_data.fillna(0, inplace=True)

In [34]:
# imputation to fix tie games (wins go to the away team here)
hist_data.loc[(hist_data['Result'] == 'T') & (hist_data['Away?'] == 1), 'Result'] = 'W'
hist_data.loc[(hist_data['Result'] == 'T') & (hist_data['Away?'] == 0), 'Result'] = 'L'

In [35]:
feature_only_data = hist_data.drop(columns=['Team', 'Date', 'Day', 'G#', 'Week', 'Opp', 'Away?', 'Result', 'Seasonweek'])
feature_target_data = hist_data.drop(columns=['Team', 'Date', 'Day', 'G#', 'Week', 'Opp', 'Away?', 'Seasonweek'])

In [36]:
corr_matrix = feature_only_data.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

threshold = 0.85

to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print(to_drop)

['Att', 'Sk Yds', 'Penalty Yds', 'FGM', 'Punt Yds', 'Def Sack Yds', 'Yards Bef Contact']


In [37]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(feature_only_data, hist_data['Result'])
importances = pd.Series(model.feature_importances_, index=feature_only_data.columns)
top_20 = importances.sort_values(ascending=False).head(20).index.tolist()

print(top_20)

['Rate', 'Def Passer Rate', 'Def Rush Att', 'Rush Att', 'TD allowed', 'Def Cmp%', '3D%', 'Def Sack Yds', 'TO', 'Sk Yds', '1stD', 'Opp3D%', 'Def Prss', 'Yards After Contacts', 'IAY', 'Pressured', 'Rush Yds', 'BadTh', 'YAC', 'Rush 1st Allowed']


In [38]:
columns_to_keep = top_20 + ['Seasonweek', 'Result', 'Team']
final_hist_data = hist_data[columns_to_keep]
final_hist_data

,Rate,Def Passer Rate,Def Rush Att,Rush Att,TD allowed,Def Cmp%,3D%,Def Sack Yds,TO,Sk Yds,...,Yards After Contacts,IAY,Pressured,Rush Yds,BadTh,YAC,Rush 1st Allowed,Seasonweek,Result,Team
0,112.0,58.1,18,25,1,70.7,90.0,49,4,5,...,38,227,2,121,3,109,3,20221,W,BUF
1,63.1,111.3,25,18,4,83.9,46.2,5,3,49,...,17,239,15,52,7,75,7,20221,L,RAM
2,94.7,144.2,27,22,6,76.9,25.0,0,0,26,...,53,267,27,103,4,89,7,20221,L,CRD
3,79.7,106.7,19,38,3,67.6,38.5,35,2,0,...,53,282,3,201,8,78,5,20221,L,ATL
4,98.3,73.9,17,21,1,62.7,38.5,12,1,2,...,7,360,4,63,7,64,3,20221,W,RAV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1625,137.8,95.5,21,24,2,65.9,54.5,15,0,13,...,32,215,5,126,2,111,3,202418,W,SEA
1626,86.0,128.5,27,28,6,71.4,38.5,0,3,28,...,62,329,9,138,8,69,8,202418,L,SFO
1627,93.4,84.2,19,31,1,61.9,62.5,12,1,5,...,45,258,5,179,4,35,4,202418,W,TAM
1628,105.4,92.4,29,32,3,64.3,33.3,11,1,16,...,25,299,5,85,4,77,8,202418,L,OTI


In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
final_hist_data[top_20] = scaler.fit_transform(final_hist_data[top_20])

C:\Users\danbi\AppData\Local\Temp\ipykernel_20332\1054485349.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_hist_data[top_20] = scaler.fit_transform(final_hist_data[top_20])


In [40]:
final_hist_data.to_csv('C:/Users/danbi/Desktop/final_hist_data.csv', index=False)